# Circuit for Shor’s algorithm using 2n+3 qubits

This is a jupyter notebook building up the controlled U-a gate needed to implement Shor's algorithm as described by Stephane Beauregard in https://arxiv.org/abs/quant-ph/0205095v3.

In [ ]:
import numpy as np
import qiskit as qk

## Gate Functions
Running this import adds each of the gates to the QuantumCircuit class.
* QFT, iQFT
* PhiADDa (No control, Single control, double control)
* PhiADDaModN
* CMULTaModN
* Cua

In [ ]:
import customGates

In [ ]:
def init_reg(circ, q, val):
    """Initializes qubit register to value with NOT gates."""
    for i in range(len(q)):
        if val & (1<<i):
            circ.x(q[i])

## Circuits

In [ ]:
from qiskit.tools.visualization import plot_histogram
sim_backend = qk.BasicAer.get_backend('qasm_simulator')

### Circuit for quantum adder

In [ ]:
b = qk.QuantumRegister(4, 'b')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(b, res_b)

# Add 3 + 11
a = 3
init_reg(circ, b, 11)

circ.qft(b, 4)
circ.PhiADDa(a, b, 4)
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Controlled Quantum Adder

In [ ]:
b = qk.QuantumRegister(4, 'b')
c = qk.QuantumRegister(1, 'c')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(b, c, res_b)

# Add 3 + 11
a = 3
init_reg(circ, b, 11)
init_reg(circ, c, 1)  # C needs to be 1 in order to add, otherwise measurement is b passed through

circ.qft(b, 4)
circ.cPhiADDa(a, b, c, 4)
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Double Controlled Quantum Adder

In [ ]:
c = qk.QuantumRegister(2, 'c')
b = qk.QuantumRegister(4, 'b')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, b, res_b)

# Add 3 + 11
a = 3
init_reg(circ, b, 11)
init_reg(circ, c, 3)  # C needs to be 3 in order to add, otherwise measurement is b passed through

circ.qft(b, 4)
circ.ccPhiADDa(a, b, c[0], c[1], 4)
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Controlled Quantum Subtractor

In [ ]:
b = qk.QuantumRegister(4, 'b')
c = qk.QuantumRegister(1, 'c')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(b, c, res_b)

# Sub 11 - 3
a = 3
init_reg(circ, b, 11)
init_reg(circ, c, 1) # C needs to be 1 in order to add, otherwise measurement is b passed through

circ.qft(b, 4)
circ.cPhiSUBa(a, b, c, 4)
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Modulo Quantum Adder

In [ ]:
c = qk.QuantumRegister(2, 'c')
b = qk.QuantumRegister(4, 'b')
anc = qk.QuantumRegister(1, 'anc')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, b, anc, res_b)

# Add (7 + 5) % 10
a = 7
N = 10
init_reg(circ, b, 5)
init_reg(circ, c, 3)  # C needs to be 3 in order to add, otherwise measurement is b passed through

circ.qft(b, 4)
circ.barrier()
circ.PhiADDaModN(a, b, c[0], c[1], anc[0], N, 4)
circ.barrier()
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Modulo Quantum Subtractor?

In [ ]:
c = qk.QuantumRegister(2, 'c')
b = qk.QuantumRegister(4, 'b')
anc = qk.QuantumRegister(1, 'anc')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, b, anc, res_b)

# Sub (2 - 7) % 10
a = 7
N = 10
init_reg(circ, b, 2)
init_reg(circ, c, 3)  # C needs to be 3 in order to add, otherwise measurement is b passed through

circ.qft(b, 4)
circ.PhiSUBaModN(a, b, c[0], c[1], anc[0], N, 4)
circ.iqft(b, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Multiply Modulo Circuit

In [ ]:
c = qk.QuantumRegister(1, 'c')
x = qk.QuantumRegister(4, 'x')
b = qk.QuantumRegister(4, 'b')
anc = qk.QuantumRegister(1, 'anc')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, x, b, anc, res_b)

# Multiply 1 + 3*4 % 7
a = 3
N = 7
init_reg(circ, x, 4)
init_reg(circ, b, 1)
init_reg(circ, c, 1)  # C needs to be 1 in order to add, otherwise measurement is b passed through

# Barriers around custom gate spaces out the drawing so they don't overlap
circ.barrier()
circ.CMULTaModN(a, b, c[0], x, anc[0], N, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Divide Modulo Circuit?
Not really a divider but the inverse of the multply

In [ ]:
c = qk.QuantumRegister(1, 'c')
x = qk.QuantumRegister(4, 'x')
b = qk.QuantumRegister(4, 'b')
anc = qk.QuantumRegister(1, 'anc')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, x, b, anc, res_b)

# Divide? (6 - 3*4) % 7
a = 3
N = 7
init_reg(circ, x, 4)
init_reg(circ, b, 6)
init_reg(circ, c, 1)  # C needs to be 1 in order to add, otherwise measurement is b passed through

# Barriers around custom gate spaces out the drawing so they don't overlap
circ.barrier()
circ.CDIVaModN(a, b, c[0], x, anc[0], N, 4)
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Controlled Swap Registers

In [ ]:
c = qk.QuantumRegister(1, 'c')
x = qk.QuantumRegister(4, 'x')
b = qk.QuantumRegister(4, 'b')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, x, b, res_b)

init_reg(circ, x, 14)
init_reg(circ, b, 3)
init_reg(circ, c, 0)  # C needs to be 1 in order to add, otherwise measurement is b passed through

# Barriers around custom gate spaces out the drawing so they don't overlap
circ.barrier()
circ.cswap(c, x, b)
circ.barrier()
circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Controlled U-a

In [ ]:
c = qk.QuantumRegister(1, 'c')
x = qk.QuantumRegister(4, 'x')
z = qk.QuantumRegister(4, 'z')
anc = qk.QuantumRegister(1, 'anc')
res_x = qk.ClassicalRegister(4, 'res\_x')

circ = qk.QuantumCircuit(c, x, z, anc, res_x)

# Multiply 3*5 % 7
a = 5
N = 7
init_reg(circ, x, 3)
init_reg(circ, c, 1)  # C needs to be 1 in order to add, otherwise measurement is x passed through

# Barriers around custom gate spaces out the drawing so they don't overlap
circ.barrier()
circ.cua(a, c[0], x, z, anc[0], N, 4)
circ.barrier()
circ.measure(x, res_x)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Shors Circuit with Sequential iQFT

In [ ]:
def rGate(circ, c, res, n, j):
    """Does the j-th row of the n-by-n iqft."""
    for i in range(j - 1, -1, -1):
        circ.u1(-np.pi/float(2**(i + 1)), c).c_if(res[i], 1)
    circ.h(c)

In [ ]:
n = 4
a = 2
N = 15

# Need the extra 1 qubit to hold the overflow of addition
x = qk.QuantumRegister(n + 1, 'x')
z = qk.QuantumRegister(n + 1, 'z')
c = qk.QuantumRegister(1, 'c')
anc = qk.QuantumRegister(1, 'anc')

# Gross way to create 2n classical registers
# Needed to do this to use res[i] as the control for the classical not
res = []
for i in range(2*n):
    res.append(qk.ClassicalRegister(1, 'res%d' % i))

circ = qk.QuantumCircuit(c, x, z, anc, *res)

# Either 2**n - 1 or 2**(n+1) - 1
# The paper shows the registion initialized to all 1s but doesn't specify if the
# addition overflow qubit is included.
init_reg(circ, x, 2**(n) - 1)

for i in range(2*n):
    circ.h(c)
    circ.cua(a**(2**i) % N, c[0], x, z, anc[0], N, n + 1)
    rGate(circ, c, res, 2*n, i)
    circ.measure(c, res[i])
    
    circ.reset(c)
    # Some papers have this classically controlled not, others don't
    circ.x(c).c_if(res[i], 1)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)